## Mega geile Klärung zum Thema Batch Normalization
<br>
https://github.com/udacity/deep-learning/blob/master/batch-norm/Batch_Normalization_Lesson.ipynb

In [ ]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow import keras


def load(data_path):
    with open(data_path,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

class MnistData:
    def __init__(self, filenames, need_shuffle, datatype='training'):
        all_data = []
        all_labels = []
        x_train, y_train, x_test, y_test = load(filenames) #"data/mnist.pkl"
        
        if datatype=='training':
            self._data = x_train / 127.5 -1
            self._labels = y_train
            print(self._data.shape)
            print(self._labels.shape)
        else:
            self._data = x_test / 127.5 -1
            self._labels = y_test
            print(self._data.shape)
            print(self._labels.shape)
            
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

In [ ]:
data = "../4_basic_image_recognition/data/mnist.pkl"
train_data = MnistData(data, True, 'training')
test_data = MnistData(data, False, 'test')

## Batch Normalization
<br>

**Steps:**


1. add convolutional layer with out activation function <br>
2. use tf.layers.batch_normalization(your_conv_layer, training=True) <br>
3. assign activition function for results

In [ ]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28*28], name='x-intput')
    y = tf.placeholder(tf.int64, [None], name='y-input')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x-input-image')

# for tensorboard image visualization
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1]) # see x.shape

with tf.name_scope('layers'):
    # add a new convolutional layer
    conv_1 = tf.layers.conv2d(inputs=x_image, 
                              filters=32, 
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=None,
                              name= 'conv1')
    # add batch normalization
    conv_1 = tf.layers.batch_normalization(conv_1, training=True)
    # use activation function
    conv_1 = tf.nn.relu(conv_1)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv_1,
                                    pool_size=(2, 2),
                                    strides=(2,2),
                                    name='pool1')

    conv_2 = tf.layers.conv2d(inputs=pool1,
                              filters=64,
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=None,
                              name='conv2')
    
    conv_2 = tf.layers.batch_normalization(conv_2, training=True)
    conv_2 = tf.nn.relu(conv_2)

    pool2 = tf.layers.max_pooling2d(inputs=conv_2,
                                    pool_size=(2,2),
                                    strides=(2,2),
                                    name= 'pool2')

    # fc layer1
    flatten = tf.layers.flatten(pool2, name= 'fc1_flatten')
    # fc layer2
    y_ = tf.layers.dense(flatten, 10, name= 'fc2_flatten')

with tf.name_scope('loss'):
    #sparse_softmax_cross_entropy include one_hot encoding
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

with tf.name_scope('accuracy'):
    
    predict = tf.argmax(y_, 1)
    correct_prediction = tf.equal(predict, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    
    with tf.name_scope('accuracy'):
        tf.summary.scalar('accuracy', accuracy)
    
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 50


# train 10k: %
with tf.Session() as sess:
    sess.run(init)
    # start training
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
                
        # print train kpi
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        
        # perform test and test kpi
        if (i+1) % 1000 == 0:
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
